In [1]:
!pip install boruta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.7 MB/s 


In [ ]:
#@title Default title text
# import pandas as pd
# import numpy as np

# lvl1_train = pd.read_csv('/content/drive/MyDrive/Sirisha_model_check/BD1/Level1/BD1_training_set.csv')
# lvl1_indep = pd.read_csv('/content/drive/MyDrive/Sirisha_model_check/BD1/Level1/BD1_training_set.csv')
# lvl2_train = pd.read_csv('/content/drive/MyDrive/Sirisha_model_check/BD1/level2/BD1_training_set_L2.csv')
# lvl2_indep = pd.read_csv('/content/drive/MyDrive/Sirisha_model_check/BD1/level2/BD1_independent_set_L2.csv')

# sub_lvl1_train = lvl1_train.groupby(['nuclear_receptor'], as_index=False, group_keys=False).apply(lambda x: x.sample(min(50, len(x))))
# sub_lvl1_indep = lvl1_indep.groupby(['nuclear_receptor'], as_index=False, group_keys=False).apply(lambda x: x.sample(min(50, len(x))))
# sub_lvl2_train = lvl2_train.groupby(['nucelar receptor'], as_index=False, group_keys=False).apply(lambda x: x.sample(min(16, len(x))))
# sub_lvl2_indep = lvl2_indep.groupby(['nuclear_receptor'], as_index=False, group_keys=False).apply(lambda x: x.sample(min(16, len(x))))

# sub_lvl1_train.to_csv('/content/drive/MyDrive/Sirisha_model_check/sub_sample/lvl1_train.csv')
# sub_lvl1_indep.to_csv('/content/drive/MyDrive/Sirisha_model_check/sub_sample/lvl1_indep.csv')
# sub_lvl2_train.to_csv('/content/drive/MyDrive/Sirisha_model_check/sub_sample/lvl2_train.csv')
# sub_lvl2_indep.to_csv('/content/drive/MyDrive/Sirisha_model_check/sub_sample/lvl2_indep.csv')

In [ ]:
#@title Default title text
# import numpy as np
# import math
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
# from hyperopt.pyll.base import scope
# from sklearn.model_selection import cross_val_score


# def hyperparameter_tune(X_train,y_train):
    
#     # Set up space dictionary with specified hyperparameters
#     space = {'n_estimators': hp.uniform('n_estimators', 75,150),
#             'max_depth': hp.uniform('max_depth', 1,100),'min_samples_leaf': hp.uniform('min_samples_leaf', 1,10),
#             'min_samples_split': hp.uniform('min_samples_split', 2,10),'max_samples': hp.uniform('max_samples',0,1),
#             'warm_start': hp.choice('warm_start',[True, False])}
    
#     # Set up objective function
#     def objective(params):
#         params = {'n_estimators': int(params['n_estimators']),
#                 'max_depth': int(params['max_depth']),'min_samples_leaf': int(params['min_samples_leaf']),
#                 'min_samples_split': int(params['min_samples_split']),'max_samples': params['max_samples'],
#                 'warm_start': params['warm_start']
#                 }
#         # scoring method can be 'accuracy','f1','ROC','MCC' etc
#         clf = RandomForestClassifier(class_weight="balanced", **params) 
#         best_score = cross_val_score(clf, X_train, y_train, scoring='f1', cv=7, n_jobs=-1).mean()
#         loss = 1 - best_score
#         return loss
    
#     # Run the algorithm - test max evals
#     best = fmin(fn=objective,space=space, max_evals=150, algo=tpe.suggest)

#     max_depth = int(best['max_depth'])
#     max_samples = best['max_samples']
#     min_samples_leaf = int(best['min_samples_leaf'])
#     min_samples_split = int(best['min_samples_split'])
#     n_estimators = int(best['n_estimators'])
#     warm_start = int(best['warm_start'])
    
#     if warm_start == 0:
#         warm_start = 'True'
#     elif warm_start == 1:
#         warm_start = 'False'
        
#     tuned_clf = RandomForestClassifier(class_weight="balanced",
#                                     max_depth= max_depth,
#                                     max_samples= max_samples,
#                                     min_samples_leaf= min_samples_leaf,
#                                     min_samples_split= min_samples_split,
#                                     n_estimators= n_estimators,
#                                     warm_start= False)
    
#     return tuned_clf

In [ ]:
#@title Default title text
# from boruta import BorutaPy
# from sklearn.ensemble import RandomForestClassifier
# import pandas as pd

# def feature_rank(df):

#   #assign features as X and label as y 
#   X = df.drop(['nuclear_receptor'],axis=1)
#   y = df['nuclear_receptor']

#   #initialize Random Forest clasifier
#   clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
#   clf.fit(X, y)

#   feature_selector = BorutaPy(clf, n_estimators='auto', verbose=2, random_state=1)
#   feature_selector.fit(X, y)

#   feature_ranks = list(zip(df.columns,feature_selector.ranking_,feature_selector.support_))

#   selected_feat_df = pd.DataFrame(feature_ranks,columns =['Feature','Ranking','Important?'])

#   return selected_feat_df


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Sirisha_model_check/GItHub')
from feature_ranking import *
from hyperparameter_tune import *
from utilities import *
from model_eval import *
from RF_model import *

# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--feature_selection', type=int, required=False, default= 1)
# parser.add_argument('--hyperparameter_tune', type=int, required=False, default= 1)
# args = parser.parse_args()

In [3]:
#@title Default title text
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score,roc_auc_score,precision_score, recall_score
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
import seaborn as sns

#evaluation metrics
#takes list of true label, list of predicted label and model that is used to predict labels
#returns evaluation score and saves confusion matrix to output directory
def eval_metrics(true,pred,rf_model):

  accuracy = accuracy_score(true,pred)
  precision = precision_score(true,pred,pos_label='positive',average='macro')
  recall = recall_score(true,pred,pos_label='positive',average='macro')
  f1 = f1_score(true,pred,pos_label='positive',average='macro')
  auc = roc_auc_score(true,pred)
  mcc = matthews_corrcoef(true,pred)

  matrix = confusion_matrix(true,pred)

  #creates confusion matrix
  plt.rcParams["figure.figsize"] = (20,8)
  sns.set(font_scale=1.4)
  sns.heatmap(matrix, annot=True, annot_kws={'size':10},
              cmap=plt.cm.Greens, linewidths=0.2,fmt='g')
  plt.xlabel('Predicted label')
  plt.ylabel('True label')

  plt.savefig('confusion_matrix.png')

  #save evaluation metrics of the model to text file
  output_file = open("model_evaluation.txt","w")

  output_file.write("Accuracy: " + str("{:.4f}".format(accuracy)) + "\n")
  output_file.write("Presicion: " + str("{:.4f}".format(precision)) + "\n")
  output_file.write("Recall: " + str("{:.4f}".format(recall)) + "\n")
  output_file.write("F1-macro: " + str("{:.4f}".format(f1)) + "\n")
  output_file.write("AUC score: " + str("{:.4f}".format(auc)) + "\n")
  output_file.write("Matthews Correlation Coefficient: " + str("{:.4f}".format(mcc)) + "\n")

  output_file.close() 

  return None

In [13]:
#@title Default title text
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score,roc_auc_score,precision_score, recall_score
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#takes train and independent dataset and returns combination of them
def combined_data(train,indep):
  train_df = pd.read_csv(train,index_col=0)
  train_df = train_df.drop(['protein_name'],axis=1)

  indep_df = pd.read_csv(indep,index_col=0)
  indep_df = indep_df.drop(['protein_name'],axis=1)
  indep_df = indep_df.dropna(axis=0)

  combined_df = [train_df,indep_df]
  comb_train = pd.concat(combined_df)

  return comb_train 

#label mapping of HPRD and refseq dataset for level 1 model
#takes HPRD or Refseq dataset and returns dataset with newly mapped label
def HPRD_Refseq_label_map_level_1(external_dataset):
  #label dictionary of NRs sub-class and NonNRs
  label_map = {0: 1,1 : 1,2 : 1,3 : 1,4 : 1,5 : 1,6 : 1,'6' : 1,'NonNR' : 0}

  external_dataset_df = pd.read_csv(external_dataset,index_col=0)
  external_dataset_df = external_dataset_df.drop(['protein_name'],axis=1)

  #apply label dictionary to convert all NR sub-class to 1 and Non-NR to 0
  external_dataset_df['nuclear_receptor'] = external_dataset_df['nuclear_receptor'].apply( lambda x : label_map[x])

  return external_dataset_df

#label mapping of combined dataset for level 2 model
#takes combined dataset and returns dataset with newly mapped label
def Combined_dataset_label_map_level_2(combined_dataset):
  #label dictionary of NRs sub-class (0-6)
  label_map = {'NR1_thyroid_hormone_like':1, 'NR2_HNF4_like':2, 'NR3_estrogen_like':3,
       'NR4_nerve_growth_factor_IB-like':4, 'NR5_fushi_tarazu-F1_like':5,
       'NR6_germ_cell_nuclear_factor_like':6, 'NR0_knirps_and_DAX_like':0}

  #apply label dictionary to convert all NR sub-class to 1 and Non-NR to 0
  combined_dataset['nuclear_receptor'] = combined_dataset['nuclear_receptor'].apply( lambda x : label_map[x])

  return combined_dataset

#train and test spli
#takes dataset and returns test and train set
def train_test_split(dataset):

  X = dataset.drop(['nuclear_receptor'],axis=1).values
  y = dataset['nuclear_receptor'].values

  return train_test_split(X, y, test_size = 0.3,stratify=y)

#get index of HPRD and refseq dataset in order to filter True Positive prediction only
#takes external dataset and predicted label of external dataset
#return external dataset will True Positive label only
def select_true_positive_only(external_dataset,external_dataset_pred):
  dataset_index = external_dataset['nuclear_receptor']
  #zip true, pred label with index and select only true positive
  pred_idx = list(zip(dataset_index.index.tolist(),dataset_index.tolist() ,external_dataset_pred.tolist()))
  true_positive_idx = [idx for idx,true,pred in pred_idx if pred!=0 and true==pred]
  #select data points with True Positive prediction only
  true_positive_label_only = external_dataset.iloc[true_positive_idx]

  return true_positive_label_only

In [12]:
#@title Default title text
from feature_ranking import *
from hyperparameter_tune import *
from utilities import *
from model_eval import *
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sys

def RF_model(a,b,combined_df,hprd_df,refseq_df):
  if a == 1:
    #rank most important feature using Borutapy feature selection method
    #this function return the list of important features
    selected_feat = feature_rank(combined_level1_df)
    #create a subset of Combined, HPRD and Refseq dataset containg only important features
    combined_imp_feat_df = combined_level1_df[np.intersect1d(combined_level1_df.columns, selected_feat)]
    hprd_imp_feat_df = hprd_df[np.intersect1d(hprd_df.columns, selected_feat)]
    refseq_imp_feat_df = refseq_df[np.intersect1d(refseq_df.columns, selected_feat)]

  else:
    #if user have choosen not to perform feature selection, then all features will be used for model training
    combined_imp_feat_df = combined_level1_df
    hprd_imp_feat_df = hprd_df
    refseq_imp_feat_df = refseq_df

  X = combined_imp_feat_df.drop(['nuclear_receptor'],axis=1).values
  y = combined_imp_feat_df['nuclear_receptor'].values

  #if user have choosen to perform hyper parameter tuning, then hyperparameters of Random Forest model will be tuned using Hyperopt python package
  #if args.hyperparameter_tune == 1:
  hprd_pred = []
  refseq_pred = []

  if b == 1:

    tuned_RF_model = hyperparameter_tune(X,y).fit(X,y)
    #predict HPRD label using base RF model
    hprd_tuned_model_pred = tuned_RF_model.predict(hprd_imp_feat_df.drop(['nuclear_receptor'],axis=1).values)
    refseq_tuned_model_pred = tuned_RF_model.predict(refseq_imp_feat_df.drop(['nuclear_receptor'],axis=1).values)

    #get tuned model evaluation on HPRD and Refseq dataset
    eval_metrics(hprd_imp_feat_df['nuclear_receptor'].values,hprd_tuned_model_pred, tuned_RF_model)
    eval_metrics(refseq_imp_feat_df['nuclear_receptor'].values,refseq_tuned_model_pred,tuned_RF_model)
    
    hprd_pred = hprd_tuned_model_pred
    refseq_pred = refseq_tuned_model_pred   
    
  else:
    #if user have not choose to perform hyper parameter tuning, then Random Forest model with default hyperparameters will be trained
    base_RF_model = RandomForestClassifier()
    base_RF_model.fit(X, y)
    #predict HPRD label using base RF model
    hprd_base_model_pred = base_RF_model.predict(hprd_imp_feat_df.drop(['nuclear_receptor'],axis=1).values)
    refseq_base_model_pred = base_RF_model.predict(refseq_imp_feat_df.drop(['nuclear_receptor'],axis=1).values)

    #get base model evaluation on HPRD and Refseq dataset
    eval_metrics(hprd_imp_feat_df['nuclear_receptor'].values,hprd_base_model_pred,base_RF_model)
    eval_metrics(refseq_imp_feat_df['nuclear_receptor'].values,refseq_base_model_pred,base_RF_model)

    hprd_pred = hprd_tuned_model_pred
    refseq_pred = refseq_tuned_model_pred 
  
  return hprd_pred, refseq_pred

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('/content/drive/MyDrive/Sirisha_model_check/GItHub') #for test only

from RF_model import *
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--feature_selection', type=int, required=False, default= 1)
# parser.add_argument('--hyperparameter_tune', type=int, required=False, default= 1)
# args = parser.parse_args()

a = 1
b = 1

#read train,independent,hprd and refseq dataset
train_lvl1 = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_lvl1_train_dataset.csv'
indep_lvl1 = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_lvl1_independent_dataset.csv'
train_lvl2 = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_lvl2_train_dataset.csv'
indep_lvl2 = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_lvl2_independent_dataset.csv'

hprd = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_HPRD_dataset.csv'
refseq = '/content/drive/MyDrive/Sirisha_model_check/GItHub/Data/subset_of_Refseq_dataset.csv'

#combine train and indep dataset to train level-1 model
combined_level1_df = combined_data(train_lvl1,indep_lvl1)

#apply label map to convert all NR sub-class to 1 and Non-NR to 0
hprd_df = HPRD_Refseq_label_map_level_1(hprd)
refseq_df = HPRD_Refseq_label_map_level_1(refseq)

#-------------------LEVEL-1 Model (NR vs NRs prediction)------------------------
#if user have choosen to perform feature selection, then importtant features will be selected using Borutapy python package
#if user have choosen not to perform feature selection, then all features will be used for model training
#if user have choosen to perform hyper parameter tuning, then hyperparameters of Random Forest model will be tuned using Hyperopt python package
#if user have not choose to perform hyper parameter tuning, then Random Forest model with default hyperparameters will be trained

#train RF model using combined dataset and test on HPRD and Refseq dataset
#following level will also returns list of predicted label for HPRD and refseq dataset
#evaluation results(confusion matrix and result table) will be saved inside 'Output' folder
print('Training and Testing Level-1 RF model')
hprd_pred_level_1,refseq_pred_level_1 = RF_model(a,b,combined_level1_df,hprd_df,refseq_df)

#-------------------LEVEL-2 Model (sub-NRs (NR0,NR1,NR2,NR3,NR4,NR5 and NR6) prediction)------------------------
#combine train and indep dataset to train level-1 model
combined_level2_df = combined_data(train_lvl2,indep_lvl2)

#apply label map to convert all NR sub-class to 0-6
combined_level2_df = Combined_dataset_label_map_level_2(combined_level2_df)

#reread HPRD and Refseq dataset without label mapping
hprd_df = pd.read_csv(hprd,index_col=0)
refseq_df = pd.read_csv(refseq,index_col=0)

#get index of HPRD and refseq dataset in order to filter True Positive prediction only
hprd_TP_only = select_true_positive_only(hprd_df,hprd_pred_level_1)
refseq_TP_only = select_true_positive_only(refseq_df,refseq_pred_level_1)

#train RF model using combined dataset and test on HPRD and Refseq dataset
#following level will also returns list of predicted label for HPRD and refseq dataset
#evaluation results(confusion matrix and result table) will be saved inside 'Output' folder
print('Training and Testing Level-2 RF model')
RF_model(a,b,combined_level2_df,hprd_TP_only,refseq_TP_only)
print('Complete !!!')